@author: Kyle Cochran

Intro:
======
Hi there. Cloud predict is a simple (or maybe not so simple) tool for improving cloud coverage predictions by using Planet Labs imagery. This can be done a whole bunch of ways, [outlined here](https://docs.google.com/document/d/1kd3TzlZg3vgJ8R7Jv20ztRru72tKuW-nnQ6g78knWHM/edit#heading=h.spyidnqefqz8). Two ways are implemented in the `cloudpredict.py` Python file. Examples of how to run each are given below.

If any of this is difficult to run as a notebook (because planet dependencies), the 'main' function inside `cloudpredict.py` has these examples and is easier to run inside of Docker.

For those interested in continuing, the code is marked up with `TODO:` statements in areas that could still use work.

In [2]:
import cloudpredict #main file
import plotmat # uber-simple matrix loader, colorer, and plotter I made
import planetingress as plani # Used for retrieving Planet data + some helper functions

import numpy as np
import matplot.pyplot as plt
import datetime

ModuleNotFoundError: No module named 'planet'

In both cases, you first must define an area you wish to predict. The format for defining this is the same as the "geometry" sub-structure of a geojson file. You can only define one polygon at a time.

The geometry here is this region of San Francisco which is particularly interesting because half is usually clear and the other half is usually cloudy:

<img src="notebook_stuff/sf_predict_area.png">

In [ ]:
'''
    This can be any single shape, but the code internally draws a rectangle 
    around and uses that  it to make pixels easier.
'''
geoaoi = {
        "type": "Polygon",
        "coordinates": [
          [
            [
              -122.48485565185547,
              37.74004179435127
            ],
            [
              -122.43232727050781,
              37.74004179435127
            ],
            [
              -122.43232727050781,
              37.77695634643178
            ],
            [
              -122.48485565185547,
              37.77695634643178
            ],
            [
              -122.48485565185547,
              37.74004179435127
            ]
          ]
        ]
      }

    '''
        The time that we're trying to predict. 
        
        This needs to be in the future to work (for the linear predicter) and needs to be in the _nearish_ future to work well.
        
        !! : change this to be in the future : !!
    '''
    prediction_time = datetime.datetime(2019, 9, 8)

Dynamical systems PDE-FINDing method
---------------------
This method uses Planet imagery to try and reconstruct a PDE that successfully models local weather patterns as a dynamical system. Cloud coverage maps over time are extracted from Planet imagery and stacked into a 3D matrix. Numerical methods are then used to approximate derivatives and the PDE-FIND method is used to reconstruct a best-fit PDE. Theory documentation for PDE-FIND can be found [at Arxiv here](https://arxiv.org/abs/1609.06401) and the open source code implementation can be found [on github here](https://github.com/snagcliffs/PDE-FIND). A more extensive explanation of our implementation and notes on how to continue can be found [in this google doc](https://docs.google.com/document/d/1r-uGVuRmNgy-l3tDs4OEH6mRriXnRUR3QzsJah6p44I/edit#heading=h.lpxx7rdo1tvk). 

NOTE: while PDE generation works well, solving arbitrary PDE's to make predictions proved to be too difficult. It is likely that this method will provide a highly accurate/precise prediction if a robust PDE solving method is devised. 


##### Generating a PDE ##### 

In [ ]:
'''
    Define the period on which to train the pde model
'''
dtstart = datetime.datetime(2019, 8, 11)
dtend= datetime.datetime(2019, 9, 11)

'''
   Find a PDE model for the weather in that area at that time.
   
   'c' is a list of coefficients (just floats)
   'description' is a list of strings that describes the terms in the PDE
   
   This function also parses the description internally and 
   saves the PDE to a file in a more machine friendly format
'''
c, description = cloudpredict.findPDE(geoaoi, dtstart, dtend)


# Print it out in a human-friendly way
cloudpredict.printPDE(c, description, ut = 'cov_t')

##### Predicting the weather with a PDE #####

This is the tricky part, and unfortunately still doesn't really work. Attempts were made using [Fenics](https://fenicsproject.org/). To see my attempts, check out the [jupyter notebook]("file://notebook_stuff/pdesim.ipynb") for that.

In [1]:

model = "path to where the model is saved locally"

'''
Uses the latest Planet imagery as initial conditions and solve the
PDE given in 'model'
'''
cloudpredict.predictPDE(geoaoi, prediction_time, model)

NameError: name 'cloudpredict' is not defined


First Order Bias and Correction Method
---------------------
This method is a much simpler approach that was written up in a week when the first method was abandoned. Starting with the base NOAA prediction for a given time, two correction steps are applied: 

#### NOAA bias drift correction ####
This step uses two data points in the past to approximate bias drift in NOAA predictions. Planet cloud cover masks are taken to be an absolute-truth model. 
Two times are taken from the past:
- *time B* should be at least a day back (this is a tuning parameter)
- *time A* should be as close to the present as possible and be at a time when Planet imagery is available


At `B`, we ask NOAA for a prediction _generated at that time_ of the cloud cover _at the newer time_. That is, we ask NOAA what they _would have given_ for a prediction of `A` if asked at `B`. We compare that to the Planet imagery to figure out how wrong the prediction was.

We then ask for another prediction of the cloud coverage at `A` given at a time that is _pretty close to_ `A`. "pretty close" is determined by the NOAA data cadence but is usually around 12 hours. We then compare _this_ prediction to the Planet imagery to determine how wrong it is. Since this prediction is not as far into the future, we expect it to be a bit better. 

The change in the "correctness" of the NOAA prediction is what I've called _bias drift_. The two bias values generated can be used to find the rate of change in bias (bias drift). This value is then used to linearly exptrapolate the expected bias into the future.

#### Local area bias correction ####
Within an region of interest, there can often be smaller scale fluctuations in cloud cover that are not accounted for by the NOAA prediction. A good example is San Francisco. The city is split by a mountain range into the "usually cloudy" western half and the "usually clear" eastern half. This step averages the pixel difference from the NOAA prediction for the past `n` Planet images. That per-pixel bias is then assumed to hold for any images in the future.

Each Planet image is binned into some smaller dimension with bigger pixels (for computations sake). The difference in cloudiness between each big pixel and the NOAA prediction is taken to create a "bias map" for each image in time. These images are averaged over time to generate one "bias map". This bias map indicates, for each pixel, how far off the truth usually is from the NOAA prediction. To get a map of the future, just add the NOAA prediction to this bias map.

Sematic NOTE: At one point we add a probability to a pixel average. This is "what is the likelyhood of this whole area being cloudy" + "what percent of this area was cloudy". This uses the interpretation of probability as the likelihood of choosing a clouding location from drawing a single point in an area and is completely valid. 


In [ ]:
# How long to look into the past to find trends
hist_days = 5

'''
    Makes the prediction
   
   returns:
       predict: (float) an prediction of the whole (boxed in) area, without counting regional effects
       predict_map: (2d Numpy array) a binned map of the (boxed in) area, counts regional effects
'''
predict, predict_map = predict_linear(geoaoi, prediction_time, history_days = hist_days)


# This saves the predict map matrix to a file so we can later check it against actual weather pix
p_time_str = plani.datetime_to_ISO8601Z(prediction_time)
now = plani.datetime_to_ISO8601Z(datetime.datetime.now())
filename = p_time_str + "_" + now + "_" + str(hist_days)
testfolder = "./tests/"
# looks like "./tests/2019-09-06T00:00:00.000000Z_2019-09-05T21:36:45.314072Z_5.npy"
np.save(testfolder + filename, predict_map)

For a prediction of : 2019-09-06T00:00:00.000000Z
Made at             : 2019-09-05T21:36:45.314072Z
Using a history of  : 5 days


original NOAA predict :		 0.00032899
corrected predict     : 	 0.14808152

The locally corrected predict maps look like:
<img src="notebook_stuff/predict_map_example.png">

The closest PlanetScope image was taken at: 2019-09-05T18:34:51.000000Z

And looks like :
<img src="notebook_stuff/planetscope_validation.png">

### Final Remarks ###

Although validation should be performed in more areas, the initial results here look promising. The linear approximation method shows significant increase in accuracy over the bare NOAA prediction for the San Francisco test case (NOAA predicted almost no clouds at all). The predicter clearly identifies the likelihood that it will be cloudier overall than the NOAA prediction, and correctly identifies the regions that are likely to be extra cloudy.

Neato, but how do prod?
========================

Glad you asked. To get this working in an automated prod-friendly way, there are a couple of things that could still be done:

##### Turn the predict maps into a single number (sort of)
To do this, we could just average over the predict map, easy. OR, we could define the region of interest as much larger than what we're actually interested in, do the prediction on that, then take our region of interest as a single pixel inside of the predict map.

##### Test in more locations
This is the obvious one, but San Francisco is sort of an edge case where I _expect_ this approach to do well. What happens in normal places?

##### Maybe make getting the NOAA weather predictions a bit faster (optional)
The NOAA weather prediction code was really only ever meant to predict the future. I had to hack it in order to sort of "predict the present from the past". Makes it kind of slow.

##### Fetch Planet data locally instead of using the public API (optional)
Grab images from the storage buckets instead of using the peoples interface. Might involve some local georectification, but would make us not have to wait for the whole "activation" then "downloading"
